In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import selenium.common.exceptions as sex

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import lxml

import time
from time import sleep
import random
import re

In [2]:
path = "C:\\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.implicitly_wait(5)

start_url = "https://www.medigatenews.com/etc/search?keyword=%EB%B0%B1%EC%8B%A0"
driver.get(start_url)

C:\Users\yukir\AppData\Local\Temp/ipykernel_15428/222092032.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [3]:
#무한으로 내려감
import time
SCROLL_PAUSE_TIME = 0.3
# 한번 스크롤 하고 멈출 시간 설정

body = driver.find_element_by_tag_name('body')
# body태그를 선택하여 body에 넣음

while True:
    last_height = driver.execute_script('return document.documentElement.scrollHeight')
    # 현재 화면의 길이를 리턴 받아 last_height에 넣음
    for i in range(4):
        body.send_keys(Keys.END)
        # body 본문에 END키를 입력(스크롤내림)
        time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script('return document.documentElement.scrollHeight')
    try:
        if new_height == last_height:
            driver.find_element_by_xpath('//*[@id="news_gisa_more_btn"]/button').click()

    except:
        print('완료')
        break;
    

C:\Users\yukir\AppData\Local\Temp/ipykernel_15428/3057496518.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  body = driver.find_element_by_tag_name('body')
C:\Users\yukir\AppData\Local\Temp/ipykernel_15428/3057496518.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="news_gisa_more_btn"]/button').click()


완료


In [26]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [5]:
urls = soup.find_all('li', attrs ={'class':'contlst thumb horizontal'} )

address_list = []
for i in urls:
    address = i.a['href']
    address = 'https://www.medigatenews.com'+address
    # if 'category' in address:
    #     pass
    # elif 'topic' in address:
    #     pass
    # else:
    #     address_list.append(address)

    address_list.append(address)
        

In [6]:
nope_list = ['인기 키워드 순위', '의사를 위한 맞춤 기사', '제약 중심 맞춤 기사','의료 중심 맞춤 기사','칼럼/MG툰',\
    'English News','자료실','사람들']

title = soup.find_all('p', class_='titxt')
title_list = []
for i in title:
    title_ = i.text
    if title_  in nope_list:
        # print(title_ )
        pass
    elif len(title_)==0:
        pass
    else:
        title_list.append(title_)

In [7]:
dates = soup.find_all('span', class_='date')
dates_list = []
for i in dates:
    dates_ = i.text
    dates_list.append(dates_)

In [8]:
print(len(address_list))
print(len(title_list))
print(len(dates_list))

1230
1230
1230


In [9]:
medi_gates = pd.DataFrame({'title':title_list, 'dates':dates_list, 'address': address_list})

In [10]:
medi_gates.head()

,title,dates,address
0,"""RNA 적용"" 큐로셀, CAR-T+면역관문억제제…바이오오케스트라, 치매 잡는 뇌세...",2022-04-13 08:45,https://www.medigatenews.com/news/999526012
1,"""난치·만성질환 극복 도전"" 엔솔바이오 골관절염 펩타이드신약·바이옴로직 황반변성 마...",2022-04-13 08:44,https://www.medigatenews.com/news/2163570408
2,1분기 국내 제약사와 VC들이 투자한 바이오사는…대기업도 전략적투자자로 적극 참여,2022-04-13 06:42,https://www.medigatenews.com/news/2090928717
3,"코로나특위, 코로나 비상대응 100일 로드맵 마련한다",2022-04-11 20:12,https://www.medigatenews.com/news/997077107
4,"의료계, 정호영 장관 내정자에 환영 ""고질적 의료 문제 해결할 적임자""",2022-04-11 05:48,https://www.medigatenews.com/news/1134765385


In [11]:
medi_gates.to_excel('medigates_v1.xlsx')

In [15]:
medi_gates.tail(5)

,title,dates,address
1225,"테라젠바이오, AI기반 환자분류 분자 아형 산출장치 'DeepOmics Marker...",2022-04-07 15:34,https://www.medigatenews.com/news/1630465467?c...
1226,고려대 메디사이언스파크에 항암신약 GMP 시설 조성,2022-04-07 13:32,https://www.medigatenews.com/news/2208421518?c...
1227,"베링거인겔하임, 탄탄한 실적으로 R&D 투자 순항",2022-04-06 16:48,https://www.medigatenews.com/news/1421179349?c...
1228,"광동제약, 비만치료제 마이크로니들 공동개발 추진",2022-04-06 10:44,https://www.medigatenews.com/news/2561316403?c...
1229,"""Moderna will try to execute our strategy to a...",2021-12-12 13:27,https://www.medigatenews.com/news/3430186822?c...


In [47]:
sub_text_list = []
text_list = []

k= 0

for i in medi_gates['address']:
    driver.get(i)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    try:
        sub_text = soup.find('p', class_ = 'subtxt').text.replace('\n','')
        if len(sub_text)>=1:
            sub_text_list.append(sub_text)
        else:
            sub_text_list.append('sub_text없음')
    except:
        sub_text_list.append('error')

    try :
        text = soup.find_all('div', class_='contarea')
        if len(text)>=1:
            text_list.append(text)
        else:
            text_list.append('text없음')
    except:
        text_list.append('error')

    k+=1
    print('---------------------------------',len(medi_gates['address'])-k, '남음---------------------------------')

    

--------------------------------- 1229 남음---------------------------------
--------------------------------- 1228 남음---------------------------------
--------------------------------- 1227 남음---------------------------------
--------------------------------- 1226 남음---------------------------------
--------------------------------- 1225 남음---------------------------------
--------------------------------- 1224 남음---------------------------------
--------------------------------- 1223 남음---------------------------------
--------------------------------- 1222 남음---------------------------------
--------------------------------- 1221 남음---------------------------------
--------------------------------- 1220 남음---------------------------------
--------------------------------- 1219 남음---------------------------------
--------------------------------- 1218 남음---------------------------------
--------------------------------- 1217 남음---------------------------------
-------------------------

In [48]:
medi_gates['부제목'] = sub_text_list
medi_gates['내용'] = text_list

In [49]:
medi_gates.head()

,title,dates,address,부제목,내용
0,"""RNA 적용"" 큐로셀, CAR-T+면역관문억제제…바이오오케스트라, 치매 잡는 뇌세...",2022-04-13 08:45,https://www.medigatenews.com/news/999526012,"[K-바이오텍 성장 전략①] 오토텔릭바이오, ASO 기반 면역표적항암제, NASH치...","[[\n, [<strong>K-바이오텍 성장 전략 </strong>, <br/>, ..."
1,"""난치·만성질환 극복 도전"" 엔솔바이오 골관절염 펩타이드신약·바이옴로직 황반변성 마...",2022-04-13 08:44,https://www.medigatenews.com/news/2163570408,[K-바이오텍 성장 전략②] 진단 분야의 다양성 확대…지노믹트리 암 분야 확장·레보...,"[[\n, [<strong>K-바이오텍 성장 전략 </strong>, <br/>, ..."
2,1분기 국내 제약사와 VC들이 투자한 바이오사는…대기업도 전략적투자자로 적극 참여,2022-04-13 06:42,https://www.medigatenews.com/news/2090928717,보타메디 해외서 8000억원 유치해 투자규모 최대…투자금 대부분 항암제 개발사 몰려,"[[\n, [\n, <figure class=""image"" style=""displa..."
3,"코로나특위, 코로나 비상대응 100일 로드맵 마련한다",2022-04-11 20:12,https://www.medigatenews.com/news/997077107,"11일 제10차 회의서 로드맵 관련 논의 진행...안철수 위원장 ""방역∙의료대응 체...","[[\n, [\n, <figure class=""image"" style=""displa..."
4,"의료계, 정호영 장관 내정자에 환영 ""고질적 의료 문제 해결할 적임자""",2022-04-11 05:48,https://www.medigatenews.com/news/1134765385,"""감염병 사태 극복 위한 경험 충분...지방의료, 기피과, 필수의료 문제 해결까지 기대""","[[\n, [\n, <figure class=""image"" style=""displa..."


In [107]:
text_list = []

for num, i in enumerate(medi_gates['내용']):
    try:
        for k in i:
            # print(k.text)
            try:
                k2 = k.text
                k2 = k2.replace('\n','.')
                text_list.append(k2)
            except AttributeError:
                # print(num)
                text_list.append(k2)
                pass
    except:
        print(num)

In [122]:
z=0
list_text = []
for i in medi_gates['내용']:
    try:
        for k in i:
            text = k.text
            list_text.append(text)
            z+=1
            pass
    except:
        print(i)
        list_text.append(i)
        # for k in i:
        #     print(k)
        # print(i)

print(z)


text없음
1229


In [140]:
text_list_v2 =[]

for i in list_text:
    i = i.split(']')[-1]
    i =i.strip()
    # i = i.split(']')[-1]
    # print(i)
    text_list_v2.append(i)

In [141]:
medi_gates['내용'] = text_list_v2

In [142]:
medi_gates.to_excel('[0413]medigate_final.xlsx')